# **VGG 19 (poribohon bd)**

**Importing librares**

In [ ]:

import numpy as np
import pandas as pd
import os
import tensorflow as tf
import keras 
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input

tf.__version__

'2.6.0'

# **GPU**

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



# **Data Generator**

**Mounting drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**slpiting in train,test and validation**

In [ ]:
!pip install split-folders tqdm

In [ ]:
import splitfolders  # or import split_folders

# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio("/content/drive/MyDrive/bd_poribohon_data", output="/content/drive/MyDrive/splited_train_test_poribohon_data", seed=1337, ratio=(.8, .1, .1), group_prefix=None) # default values


Copying files: 8184 files [45:27,  3.00 files/s]


**Data augmentation**

In [ ]:
# Part 1 - Data Preprocessing

# Preprocessing the Training set
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/splited_train_test_poribohon_data/train',
                                                 target_size = (224,224),
                                                 batch_size = 64,
                                                 shuffle = True,
                                                 class_mode = 'categorical')

# Preprocessing the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/splited_train_test_poribohon_data/test',
                                            target_size = (224, 224)
                                            )


Found 6541 images belonging to 15 classes.
Found 832 images belonging to 15 classes.


In [ ]:
training_set.image_data_generator

**Building our model without top layer**

In [ ]:
# from tensorflow.keras.applications.vgg19 import VGG19
# #vgg = VGG19(weights="../input/vgg19/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5",include_top = False,input_shape=(224,244,3))
# vgg = VGG19(weights="../input/vgg19/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5",
#             include_top = False,input_shape=(224,224,3)
#            )
# for layer in vgg.layers:
#     layer.trainable = False
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import Flatten,Dense
# model = Sequential()
# model.add(vgg)
# model.add(Flatten())
# model.add(Dense(2,activation="sigmoid"))
# model.summary()


# Part 2 - Building the CNN
from tensorflow.keras.applications.vgg19 import VGG19
input_tensor = Input(shape=(224, 224, 3))
vgg_weights = "/content/drive/MyDrive/vgg19_weight/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
base_model = tf.keras.applications.VGG19(
    input_tensor=input_tensor,
    include_top  = False,
    weights      = None,
)
base_model.load_weights(vgg_weights)
x = base_model.output
print(base_model.input)
print(base_model.output)




KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_2'), name='input_2', description="created by layer 'input_2'")
KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 512), dtype=tf.float32, name=None), name='block5_pool/MaxPool:0', description="created by layer 'block5_pool'")


# ===============After traing===========

In [ ]:
val_loss: 0.7285 - val_categorical_crossentropy: 0.7285 - val_accuracy: 0.7512

# ===============After traing===========

In [ ]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
#base_model.trainable = False
x = tf.keras.layers.GlobalAveragePooling2D()
#x = Dense(1024, activation='relu')(x)
predictions = tf.keras.layers.Dense(15,activation="softmax")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense
model = Sequential([base_model,
                   x,
                    predictions
                ])
print(model)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 7, 7, 512)         20024384  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 15)                7695      
Total params: 20,032,079
Trainable params: 20,032,079
Non-trainable params: 0
_________________________________________________________________


In [ ]:
base_model.summary()

In [ ]:
#base_model.trainable = False
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
def get_callbacks(name):
    return[
        tfdocs.modeling.EpochDots(),
        tf.keras.callbacks.EarlyStopping(monitor='val_categorical_crossentropy',patience=1),
    ]

In [ ]:
base_learning_rate  = 0
model.compile(optimizer=tf.keras.optimizers.Adam(), loss="categorical_crossentropy",metrics=[tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),'accuracy'])


In [ ]:
base_learning_rate  = 0
base_model.compile(optimizer=tf.keras.optimizers.Adam(), loss="categorical_crossentropy",metrics=[tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),'accuracy'])


**Defining checkpoint(checkpoint directory,saving frequency)**

In [ ]:
import os
checkpoint_path='drive/MyDrive/poribohon_saved_point/cp-{epoch:04d}.ckpt'
checkpoint_dir=os.path.dirname(checkpoint_path)
cp_callback=tf.keras.callbacks.ModelCheckpoint(checkpoint_path,verbose=1,save_best_only=True)

In [ ]:
history = model.fit(x = training_set,epochs= 1000, validation_data = test_set,callbacks=[cp_callback])

# **Load last checkpoint and evaluate test accuracy**

In [ ]:
# checkpoint_dir
latest=tf.train.latest_checkpoint(checkpoint_dir)
latest

In [ ]:

model.load_weights('/content/drive/MyDrive/poribohon_saved_point/cp-0314.ckpt') # here path will be latest checkpoint path

In [ ]:
model.evaluate(test_set)
# loss,acc=model.evaluate(test_set)
# print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

26/26 [==============================] - 363s 13s/step - loss: 0.7200 - categorical_crossentropy: 0.7200 - accuracy: 0.7512


[0.7200157642364502, 0.7200157642364502, 0.7512019276618958]

In [ ]:
history = model.fit(x = training_set,initial_epoch=314,epochs= 1000, validation_data = test_set,callbacks=[cp_callback])

Epoch 315/1000
  9/103 [=>............................] - ETA: 42:57 - loss: 0.7521 - categorical_crossentropy: 0.7521 - accuracy: 0.7535

# **Evaluating the model**

In [ ]:
model.load_weights('/content/drive/MyDrive/model_trained.h5')
model.evaluate(test_set)

In [ ]:
# model.history

# **Visualization**

In [ ]:
import matplotlib.pyplot as plt
# Accuracy/validation plots
h = model.history
fig = plt.figure(figsize = (13, 4))

plt.subplot(121)
plt.plot(h['accuracy'], label = 'acc')
plt.plot(h['val_accuracy'], label = 'val_acc')
plt.legend()
plt.grid(False)
plt.title(f'Training and validation accuracy')

plt.subplot(122)
plt.plot(h['loss'], label = 'loss')
plt.plot(h['val_loss'], label = 'val_loss')
plt.legend()
plt.grid(False)
plt.title(f'Training and Test loss')

In [ ]:
from tensorflow import keras
model = keras.models.load_model('/content/drive/MyDrive')

In [ ]:
model.save_weights("/content/drive/MyDrive/model_poribohon_trained.h5")

In [ ]:
tf.keras.models.save_model(
    model,
    "/content/drive/MyDrive/keras_saved_1",
    overwrite=True,
    include_optimizer=True,
    save_format='h5',
    signatures=None,
    options=None,
    save_traces=True,
)

# =====================fine tuning =================

In [ ]:
model.summary()


In [ ]:
for layer in model.layers[1:]:
  layer.trainable = False;

In [ ]:
for layer in model.layers:
  print("{}: {}".format(layer, layer.trainable))

<keras.engine.functional.Functional object at 0x7f794b0fda50>: True
<keras.layers.pooling.GlobalAveragePooling2D object at 0x7f794b08c790>: False
<keras.layers.core.Dense object at 0x7f794b08cc10>: False


In [ ]:
for layer in base_model.layers[:21]:
  print(layer)

In [ ]:
for layer in model.layers:
  if(layer.name == 'vgg19'):
    for l in layer.layers:
      print("{}: {}".format(l, l.trainable))

<keras.engine.input_layer.InputLayer object at 0x7f79532e7c50>: False
<keras.layers.convolutional.Conv2D object at 0x7f79532e7990>: False
<keras.layers.convolutional.Conv2D object at 0x7f79523242d0>: False
<keras.layers.pooling.MaxPooling2D object at 0x7f7952411390>: False
<keras.layers.convolutional.Conv2D object at 0x7f79522c1b10>: False
<keras.layers.convolutional.Conv2D object at 0x7f79522c5550>: False
<keras.layers.pooling.MaxPooling2D object at 0x7f79522ce3d0>: False
<keras.layers.convolutional.Conv2D object at 0x7f79522d1b50>: False
<keras.layers.convolutional.Conv2D object at 0x7f7952256d50>: False
<keras.layers.convolutional.Conv2D object at 0x7f79522c9e10>: False
<keras.layers.convolutional.Conv2D object at 0x7f7952260c90>: False
<keras.layers.pooling.MaxPooling2D object at 0x7f7952263790>: False
<keras.layers.convolutional.Conv2D object at 0x7f79522c5a90>: False
<keras.layers.convolutional.Conv2D object at 0x7f79522d1090>: False
<keras.layers.convolutional.Conv2D object at 0

In [ ]:
for layer in model.layers:
  if(layer.name == 'vgg19'):
    for l in layer.layers[15:]:
      l.trainable = True